In [20]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd


In [21]:
df = pd.read_csv('../../data/replaced_english_to_sinhala.csv')

import pandas as pd

# Assuming 'df' is your DataFrame
# Display up to 500 rows from the "Comment" column
comments_subset = df['Comment'].head(500)
print(comments_subset)


0                                    2000 එකෙක්ද 😂
1                              මොඩ පොලිස් gothyek😂
2                                    හොද වැඩේ..😂😂😂
3                       කොල්ලට bike එකට ආස හිතිල😅😅
4                                         ගහපිය 😂💔
                          ...                     
495                            බයික් aka hambunada
496                        ඌට පදින්න ආස හිතිලා 🥴😂👍
497                          ගහපන් අයියෙ අල්ලගෙන😂😂
498     කනපලා ගන්නේ නැතුව බැහින්න කිව්වා නම් හරි 😅
499    @Sanju D Liyanage කොහෙ ගියත් පොලිස් පකෙක් 😅
Name: Comment, Length: 500, dtype: object


In [22]:
df.describe()

,Comment Id,User Id,Reply Comment Id,Digg Count,Reply Count,Video Id
count,3.970000e+03,3.970000e+03,3.970000e+03,3970.000000,3904.000000,3.970000e+03
mean,7.221770e+18,6.953478e+18,1.199558e+17,1.136524,0.074283,7.210500e+18
std,2.123808e+16,4.107298e+17,9.226974e+17,22.970278,1.103933,1.024129e+03
min,7.210438e+18,1.690231e+15,0.000000e+00,0.000000,0.000000,7.210500e+18
25%,7.210718e+18,6.849306e+18,0.000000e+00,0.000000,0.000000,7.210500e+18
50%,7.211414e+18,6.998108e+18,0.000000e+00,0.000000,0.000000,7.210500e+18
75%,7.230551e+18,7.119436e+18,0.000000e+00,0.000000,0.000000,7.210500e+18
max,7.331623e+18,7.321315e+18,7.238263e+18,1075.000000,47.000000,7.210500e+18


In [23]:
df.isnull().sum()


Comment Id            0
User Id               0
Unique Id             0
Nickname              1
Avatar                0
User URL              0
Reply Comment Id      0
Comment               0
Digg Count            0
Reply Count          66
Is Author Digged      0
Author Pin           66
Language            420
Create Time           0
Video URL             0
Video Id              0
English_Words         0
dtype: int64

In [35]:

from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import re
import nltk
nltk.download('vader_lexicon')


def analyze_sinhala_comments(comments):
    # Initialize the sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # Define keywords associated with claims and opinions in Sinhala
    claim_keywords_sinhala = ['තීරණය', 'සාකච්ඡාව', 'ප්‍රත්‍යානය']
    opinion_keywords_sinhala = ['අදහස්', 'අනාවැක්ෂාව', 'කිව්ව']

    # Initialize counts for claim and opinion keywords
    claim_keyword_count = 0
    opinion_keyword_count = 0

    # Initialize sentiments list
    sentiments = []

    # Analyze sentiment and count keyword occurrences
    for comment in comments:
        sentiment_score = sid.polarity_scores(comment)['compound']

        # Classify comments as positive, negative, or neutral based on compound score
        if sentiment_score >= 0.05:
            sentiment = 'positive'
        elif sentiment_score <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'

        sentiments.append(sentiment)

        # Count occurrences of claim and opinion keywords in comments
        claim_keyword_count += sum(1 for keyword in claim_keywords_sinhala if re.search(rf'\b{keyword}\b', comment, re.IGNORECASE))
        opinion_keyword_count += sum(1 for keyword in opinion_keywords_sinhala if re.search(rf'\b{keyword}\b', comment, re.IGNORECASE))

    # Make a determination based on sentiment and keyword analysis
    overall_sentiment = max(set(sentiments), key=sentiments.count)
    if overall_sentiment == 'positive' and opinion_keyword_count > claim_keyword_count:
        return 'Opinion'
    elif overall_sentiment == 'negative' and opinion_keyword_count > claim_keyword_count:
        return 'Opinion'
    elif overall_sentiment == 'neutral' and opinion_keyword_count > claim_keyword_count:
        return 'Opinion'
    else:
        return 'Claim'

# Read the CSV file
df = pd.read_csv('../../data/replaced_english_to_sinhala.csv')

# Assuming 'Comment' is the column containing Sinhala comments
sinhala_comments = df['Comment']

video_type = analyze_sinhala_comments(sinhala_comments)
print(f'Sinhala Video Type: {video_type}')

[nltk_data] Downloading package vader_lexicon to C:\Users\Zenith
[nltk_data]     Anthony\AppData\Roaming\nltk_data...


Sinhala Video Type: Opinion


In [36]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# Load pre-trained sentiment analysis model
sentiment_analysis = pipeline('sentiment-analysis')

# Assuming 'Comment' is the column containing Sinhala comments
comments = df['Comment']

# Perform sentiment analysis on each comment
sentiments = sentiment_analysis(comments.tolist())

# Add the sentiment labels to the DataFrame
df['Sentiment_Label'] = [sentiment['label'] for sentiment in sentiments]
df['Sentiment_Score'] = [sentiment['score'] for sentiment in sentiments]

# Define keywords associated with claims and opinions in Sinhala
claim_keywords_sinhala = ['තීරණය', 'සාකච්ඡාව', 'ප්‍රත්‍යානය']
opinion_keywords_sinhala = ['අදහස්', 'අනාවැක්ෂාව', 'කිව්ව']

# Count occurrences of claim and opinion keywords in comments
df['Claim_Keyword_Count'] = df['Comment'].apply(lambda x: sum(1 for keyword in claim_keywords_sinhala if re.search(rf'\b{keyword}\b', x, re.IGNORECASE)))
df['Opinion_Keyword_Count'] = df['Comment'].apply(lambda x: sum(1 for keyword in opinion_keywords_sinhala if re.search(rf'\b{keyword}\b', x, re.IGNORECASE)))

# Make a determination based on sentiment and keyword analysis
df['Video_Type'] = 'Claim'
df.loc[(df['Sentiment_Label'] == 'POSITIVE') & (df['Opinion_Keyword_Count'] > df['Claim_Keyword_Count']), 'Video_Type'] = 'Opinion'
df.loc[(df['Sentiment_Label'] == 'NEGATIVE') & (df['Opinion_Keyword_Count'] > df['Claim_Keyword_Count']), 'Video_Type'] = 'Opinion'
df.loc[(df['Sentiment_Label'] == 'NEUTRAL') & (df['Opinion_Keyword_Count'] > df['Claim_Keyword_Count']), 'Video_Type'] = 'Opinion'

# Display the resulting DataFrame
print(df[['Comment', 'Sentiment_Label', 'Sentiment_Score', 'Claim_Keyword_Count', 'Opinion_Keyword_Count', 'Video_Type']])


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\Zenith Anthony\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zenith Anthony\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd

# Load your CSV file
df = pd.read_csv('../../data/replaced_english_to_sinhala.csv')

# Assuming 'Comment' is the column containing Sinhala comments
comments = df['Comment']

# Load pre-trained BERT model and tokenizer for sentiment analysis
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Tokenize and predict sentiment for each comment
sentiments = []
for comment in comments:
    inputs = tokenizer(comment, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax().item()
    sentiments.append(predicted_class)

# Determine overall sentiment
positive_count = sum(1 for sentiment in sentiments if sentiment == 2)  # BERT sentiment scale: 0-negative, 1-neutral, 2-positive
negative_count = sum(1 for sentiment in sentiments if sentiment == 0)

# Make a determination based on overall sentiment
if positive_count > negative_count:
    overall_sentiment = 'positive'
elif positive_count < negative_count:
    overall_sentiment = 'negative'
else:
    overall_sentiment = 'neutral'

# Output the result
print(f'Overall Sinhala Sentiment: {overall_sentiment}')


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

c:\Users\Zenith Anthony\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zenith Anthony\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Overall Sinhala Sentiment: positive
